In [2]:
from Pricing.arbitrage_check import arbitrage
from Pricing.Data_processor import data_reader,set_label

In [3]:
data = data_reader('market_data.xlsx','Swaptions data')
labels = set_label(data['strike_spreads'],data['num_strikes'],data['expiries'],data['tenors'])
[beta,rho,nu]=[0.5,0,0.001]

In [5]:
arbitrage(0.001,data['F'],data['K'],data['expiries'],0,1,0.0001,'Hagan','lognormal')

('SABR volatility method:', 'Hagan')
('volatility distribution:', 'lognormal')
('Expiry=', 0.25, 'Strike price=', 0.001, 'p.d.f=', 0.0, 'Yes')
('Expiry=', 0.25, 'Strike price=', 0.006000000000000001, 'p.d.f=', 0.0, 'Yes')
('Expiry=', 0.25, 'Strike price=', 0.011000000000000001, 'p.d.f=', 7788.0632246115492, 'No')
('Expiry=', 0.25, 'Strike price=', 0.013500000000000002, 'p.d.f=', 7788.0078307140939, 'No')
('Expiry=', 0.25, 'Strike price=', 0.016, 'p.d.f=', 7788.0078307140939, 'No')
('Expiry=', 0.25, 'Strike price=', 0.018500000000000003, 'p.d.f=', 7788.0078307139211, 'No')
('Expiry=', 0.25, 'Strike price=', 0.021000000000000001, 'p.d.f=', 7788.0078307140939, 'No')
('Expiry=', 0.25, 'Strike price=', 0.025999999999999999, 'p.d.f=', 7788.0078307137474, 'No')
('Expiry=', 0.25, 'Strike price=', 0.031000000000000003, 'p.d.f=', 7788.0078307144413, 'No')
('Expiry=', 0.5, 'Strike price=', 0.001, 'p.d.f=', 0.0, 'Yes')
('Expiry=', 0.5, 'Strike price=', 0.006000000000000001, 'p.d.f=', 0.0, 'Yes')
(

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])